In [1]:
import sys
import os
from pathlib import Path

# Get absolute path to project root
project_root = Path(os.getcwd()).resolve()
if project_root.name != "ncg87-blockchain_tracker":
    project_root = project_root.parent  # Adjust if running from a subdirectory

sys.path.append(str(project_root))

# Add to Python path if not already there
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from config import Settings

from database import MongoDatabase, MongoInsertOperations, MongoQueryOperations
from database import SQLDatabase, SQLInsertOperations, SQLQueryOperations
from chains import BaseContractProcessor
from chains import BaseChainProcessor, BaseChainQuerier
import json
import web3


In [2]:
sql_db = SQLDatabase()
w3 = web3.Web3(web3.HTTPProvider(Settings.BASE_ENDPOINT))


In [3]:
query = "SELECT * FROM evm_contract_abis WHERE network = 'Base'"
sql_db.cursor.execute(query)
contract_abis = sql_db.cursor.fetchall()


In [4]:
querier = BaseChainQuerier()
mongo_db = MongoDatabase()
processor = BaseChainProcessor(sql_db, mongo_db, querier)


In [5]:
abi = processor.get_contract_abi('0x4200000000000000000000000000000000000006')


In [19]:
abi

[{'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'src',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'address',
    'name': 'guy',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'wad',
    'type': 'uint256'}],
  'name': 'Approval',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'dst',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'wad',
    'type': 'uint256'}],
  'name': 'Deposit',
  'type': 'event'},
 {'anonymous': False,
  'inputs': [{'indexed': True,
    'internalType': 'address',
    'name': 'src',
    'type': 'address'},
   {'indexed': True,
    'internalType': 'address',
    'name': 'dst',
    'type': 'address'},
   {'indexed': False,
    'internalType': 'uint256',
    'name': 'wad',
    'type': 'uint256'}],
  'name': 'Transfer',
  'type': 'event'},
 {'anonymous'

In [20]:
contract = querier.get_contract('0x4200000000000000000000000000000000000006', abi)


In [21]:
contract.functions.token0().call()


ABIFunctionNotFound: ("The function 'token0' was not found in this ", "contract's abi.")

In [22]:
hasattr(contract.token0, 'factory')


AttributeError: 'Contract' object has no attribute 'token0'

In [10]:
import asyncio
from concurrent.futures import ThreadPoolExecutor
from functools import partial

async def process_all_contracts():
    # Create a thread pool - adjust max_workers as needed
    thread_pool = ThreadPoolExecutor(max_workers=10)
    loop = asyncio.get_event_loop()
    
    # Create tasks that run in thread pool
    tasks = [
        loop.run_in_executor(
            thread_pool,
            partial(asyncio.run, processor._process_contract(contract[1], json.loads(contract[2])))
        )
        for contract in contract_abis
    ]
    
    # Gather results
    results = await asyncio.gather(*tasks)
    contract_info = [result for result in results if result]
    for result in contract_info:
        print(result)
    
    thread_pool.shutdown()
    return contract_info

# Run the async function

In [11]:
contract_info = await process_all_contracts()

Error querying EVM swap for network Base: no results to fetch
Error querying EVM token info for network Base: no results to fetch
Traceback (most recent call last):
  File "C:\Users\Nicko\vscodeprojects\blockchain_tracker\python\database\sql\query_ops.py", line 295, in query_evm_token_info
    result = self.db.cursor.fetchone()
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nicko\anaconda3\envs\blockchain\Lib\site-packages\psycopg2\extras.py", line 86, in fetchone
    res = super().fetchone()
          ^^^^^^^^^^^^^^^^^^
psycopg2.ProgrammingError: no results to fetch
Error querying EVM token info for network Base: no results to fetch
Traceback (most recent call last):
  File "C:\Users\Nicko\vscodeprojects\blockchain_tracker\python\database\sql\query_ops.py", line 295, in query_evm_token_info
    result = self.db.cursor.fetchone()
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\Nicko\anaconda3\envs\blockchain\Lib\site-packages\psycopg2\extras.py", line 86, in fetchone
 

: 